<a href="https://colab.research.google.com/github/AIKevin/Deep-Learning/blob/master/OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [0]:
class OCRModel:
  
  def __init__(self, hpm):
    self.hpm = hpm
    self.graph = tf.Graph()
    self.step = tf.train.global_step()
  
  def add_vgg(self, sess, meta_path, check_path):
    with self.graph.as_default():
      saver = tf.train.import_meta_graph(meta_path)
      with tf.Session() as sess:
        saver.restore(sess, tf.train.latest_checkpoint(check_path))
       
      self.input = self.graph.get_tensor_by_name('input:0')
      vgg_out = self.graph.get_tensor_by_name('pool5:0')
      self.vgg_out = tf.stop_gradient(vgg_out)
      
      
  def get_graph(self):
    return self.graph
  
  
  def add_placeholders(self):
    
    with self.graph.as_default():
      self.keep_prob = tf.placeholder(dtype = tf.float32)
      self.target = tf.placeholder(dtype=tf.int32, shape=[self.hpm['batch_size']])
      
  def add_fully_connected(self):
    
    with self.graph.as_default():
      fc_in = tf.reshape(self.vgg_out, [-1, 512], name='fc_in' )

      fc1_w = tf.get_variable(shape=[512, 124], dtype=tf.float32, name="fc1_w")
      fc1_b = tf.get_variable(shape=[124], dtype=tf.float32, name='fc1_b')
      fc1 = tf.identity(tf.matmul(fc_in, fc1_w) + fc1_b, 'fc1')

      dropout = tf.nn.dropout(fc1, self.keep_prob)

      fc2_w = tf.get_variable(shape=[124, self.hpm['output_num']], dtype=tf.float32, name="fc2_w")
      fc2_b = tf.get_variable(shape=[self.hpm['output_num']], dtype=tf.float32, name='fc2_b')
      fc2 = tf.identity((tf.matmul(dropout, fc2_w) + fc2_b), 'fc2')

      self.output = fc2
      
  def add_loss(self):
    with self.graph.as_default():
      target = tf.one_hot(self.target, depth = self.hpm['output_num'])
      self.loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels = target, logits = self.output, name="soft_cross_entropy")
    
  def add_train_op(self):
    with self.graph.as_default():
      adam = tf.train.AdamOptimizer()
      self.optimize = adam.minimize(self.loss, global_step = self.step)
    
  def add_decode_op(self):
    with self.graph.as_default():
      self.decoded_output = tf.argmax(tf.nn.softmax(self.output), axis=-1)
    
  def set_session(self, sess):
    self.sess = sess
    
  def _generator(self):
    feats = np.random.normal(0, 1, self.hpm['batch_size'])
    labels = np.random.normal(0, 1, self.hpm['batch_size'])
    yield feats, labels
  
  def dataread(self, data_path):
    data= pd.read_csv(data_path).values
    labels=data[:,0]
    features= data[:,1:]
    features= tf.reshape(features, (hpm['input_size'],hpm['input_size']), name= 'batch_inp')
    return features, labels
    
  def batcher(self, dataset):
    shapes= (self.hpm['batch_size'],hpm['input_size'])
    dataset = tf.data.Dataset.from_generator(generator=_generator,
                                             output_types=(tf.float32, tf.float32),
                                             output_shapes=shapes)
    dataset = dataset.batch(self.hpm['batch_size'])
    dataset = dataset.repeat(self.hpm['num_epochs'])
    iterator = dataset.make_one_shot_iterator()
    features_tensors, labels = iterator.get_next()
    return features_tensors, labels
  
  def make_feed_dict(self, batch):
    feed_dict = {}

    feed_dict[x] = batch[0]
    feed_dict[y]= batch[1]
   

    return feed_dict
  
  
  def train(self, batch):
    with self.graph.as_default():
      to_feed = self.make_feed_dict(batch)
      to_return = {'train_op': self.optimize,
                   'loss' : self.loss,
                   'global_step' : self.step}
      
      return self.sess.run(to_return, to_feed)

In [0]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config, ...)

def get_config():
  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True
  config.allow_soft_placement = True
  return config

def run_training(model, batcher, hpm, training_steps, check_dir):
  
  with tf.train.MonitoredTrainingSession(checkpoint_dir  = checkdir,
                                        hooks = [tf.train.StopAtStepHook(last_step=training_steps)],
                                        save_summaries_steps = None, save_summaries_secs= None,
                                        save_checkpoint_steps=4, scaffold=tf.train.Scaffold(saver=tf.train.Saver(max_to_keep=3)),
                                        config = get_config()) as sess:
    model.set_session(sess)
    while not sess.should_stop():
      step = tf.train.global_step(sess, model.step)
      tf.logging.info('Starting training model on a batch ...')
      t0 = time.time()
      batch = batcher.next()      
      results=model.train(batch)
      t1=time.time()
      tf.logging.info('seconds for step %d : %.3f', results['global_step'],t1-t0)
      
      loss=results['loss']
      
      tf.logging.info("loss : %f", loss)
      
def restore_model(sess, hpm, path=None):
    saver = tf.train.Saver()
    try:
      if path:
        saver.restore(sess, path)
        return True
      else:
        saver.restore(s, tf.train.latest_checkpoint(path))
        return True
    except:
      tf.logging.warning("Error restoring model")
      return False
      
  
  

In [0]:
tf.reset_default_graph()
model = OCRModel()
s = tf.Session(graph=model.get_graph())
model.add_vgg(s, '/content/gdrive/My Drive/OCR/checkpoints/model.ckpt.meta', '/content/gdrive/My Drive/OCR/checkpoints')
model.add_placeholders()
model.add_fully_connected()



INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/OCR/checkpoints/model.ckpt


<tf.Tensor 'input:0' shape=(?, 32, 32, 3) dtype=float32>

In [0]:
a = np.random.randn(10,32,32,3)
a = a.astype(np.float32)

with tf.Session(graph=model.get_graph())  as s:
  s.run(tf.global_variables_initializer())
  print(np.shape(s.run(model.output, {model.input:a, model.keep_prob:0.5})))

(10, 62)


In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import os

In [0]:
def f1(dest):

    total = 0
    for root, dirs, files in os.walk(dest):
      prin(root)


In [0]:
f1("/content/gdrive/My drive/OCR/by_class.zip")

In [0]:
data= pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/emnist-byclass-train.csv").values



In [0]:
y=data[:,0]
x= data[:,1:]